Given Twitter US Airline Sentiment Dataset, which contains data for over 14000 tweets, your task is to predict the sentiment of the tweet i.e. positive, negative or neutral.

In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk import word_tokenize 
import re

In [ ]:
training_data=pd.read_csv('training_twitter_x_y_train.csv')
training_data["text"][0]

In [ ]:
testing_data=pd.read_csv("test_twitter_x_test.csv")
testing_data["text"][0]

In [ ]:
x=training_data["text"]
type(x)

In [ ]:
xtest=testing_data["text"]
xtest=np.array(xtest)
xtest=xtest.reshape(len(xtest),1)
xtest.shape

In [ ]:
y=training_data['airline_sentiment']
x=np.array(x)
y=np.array(y)
x=x.reshape(len(x),1)
y=y.reshape(len(y),1)
x.shape,y.shape

In [ ]:
train=np.append(x,y,axis=1)
train.shape

In [ ]:
test=xtest
test.shape

In [ ]:
documents = []
c=0
for text,category in train:
    c+=1
    documents.append((word_tokenize(text), category))
documents[0:2]

In [ ]:
test_documents = []
c=0
for text in test:
    c+=1
    test_documents.append(word_tokenize(str(text)))
test_documents[0:2]

In [ ]:
import random
random.shuffle(documents)
documents[0:2]

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk import pos_tag
w = "better"
pos_tag([w])

In [ ]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)

In [ ]:
def clean_review(words):
    output_words = []
    #regex = re.compile('[a-zA-Z0-9_-]+$')
    regex = re.compile('[a-zA-Z_-]+$')
    for w in words:
        if w.lower() not in stops and len(w)>2 and not w.isnumeric() and re.match(regex,w):
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [ ]:
documents = [(clean_review(document), category) for document, category in documents]

In [ ]:
test_documents = [clean_review(document) for document in test_documents]

In [ ]:
training_documents = documents
testing_documents = test_documents

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
categories = [category for document, category in training_documents]

In [ ]:
text_documents = [" ".join(document) for document, category in training_documents]

In [ ]:
test_text_documents = [" ".join(document) for document in testing_documents]

In [ ]:
count_vec = CountVectorizer(max_features = 7000)

In [ ]:
x_train_features = count_vec.fit_transform(text_documents)
x_train_features.todense()

In [ ]:
x_train_features.shape

In [ ]:
regex = re.compile('[a-zA-Z_-]+$')
re.match(regex,'Coding')

In [ ]:
count_vec.get_feature_names()

In [ ]:
x_test_features = count_vec.transform(test_text_documents)
x_test_features

In [ ]:
xtrain=x_train_features
ytrain=categories
xtest=x_test_features

In [ ]:
import numpy as np

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svc=SVC(kernel="rbf")
grid={'C':[1e2,1e3,5e3,1e4,5e4,1e5],'gamma':[1e-3,5e-4,1e-4,5e-3]}
abc=GridSearchCV(svc,grid)
abc.fit(xtrain, ytrain)

In [ ]:
abc.best_estimator_

In [ ]:
abc.cv_results_

In [ ]:
svc = SVC(C=100,kernel='rbf',gamma=0.001)
svc.fit(xtrain, ytrain)

In [ ]:
svc=svc.predict(xtest)
svc=np.array(svc)
print(svc)
np.savetxt("twitter_pred_svc_rbf.csv",svc, fmt='%s',encoding=None)